Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре). 
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

In [1]:
!pip3 install pymystem3

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from collections import Counter
from pymystem3 import Mystem
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## 1. Загружаем данные и готовим словарь с леммами

В качестве лемматизатора мы будем использовать Mystem, потому что он работает достаточно хорошо и быстро. В него уже вшит хороший токенизатор, поэтому весь препроцессинг мы отдадим ему — регулярки нам больше не нужны.

In [4]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/word_embeddings/wiki_data.txt

--2025-12-27 15:05:37--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/word_embeddings/wiki_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68582461 (65M) [text/plain]
Saving to: ‘wiki_data.txt’

wiki_data.txt       100%[===================>]  65.41M   217MB/s    in 0.3s    

2025-12-27 15:05:38 (217 MB/s) - ‘wiki_data.txt’ saved [68582461/68582461]



In [2]:
# Препроцессинг полностью ложиться на Mystem
m = Mystem()


def preprocess(text):
    text = text.replace('#', ' ')
    lemmas = m.lemmatize(text)
    tokens = [token.strip() for token in lemmas if token.strip().isalnum()]
    return tokens

In [3]:
wiki = open('wiki_data.txt', encoding='utf8').read().split('\n')

In [4]:
# Параметр для управления размером данных
NUM_TEXTS = len(wiki)
wiki = wiki[:NUM_TEXTS]
print(f"Используется текстов: {len(wiki)}")

Используется текстов: 20003


In [8]:
vocab = Counter()

for text in tqdm(wiki):
    vocab.update(preprocess(text))

print(f"Всего уникальных слов: {len(vocab)}")

  0%|          | 0/20003 [00:00<?, ?it/s]

Всего уникальных слов: 184474


In [9]:
# Фильтруем редкие слова

freq = 30
filtered_vocab = set()

for word in vocab:
    if vocab[word] > freq:
        filtered_vocab.add(word)

print(f"Слов после фильтрации: {len(filtered_vocab)}")

Слов после фильтрации: 12618


In [10]:
word2id = {'PAD': 0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i: word for word, i in word2id.items()}

In [11]:
# Собираем предложения
sentences = []

for text in tqdm(wiki):
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    if ids:
        sentences.append(ids)

print(f"Всего предложений: {len(sentences)}")

  0%|          | 0/20003 [00:00<?, ?it/s]

Всего предложений: 19813


In [12]:
# Функция для паддинга

def pad_sequences(sequences, maxlen, padding='post', value=0):
    res = np.full((len(sequences), maxlen), value, dtype='int64')
    for i, seq in enumerate(sequences):
        if not seq:
            continue
        if len(seq) >= maxlen:
            if padding == 'post':
                res[i] = np.array(seq[:maxlen])
            else:
                res[i] = np.array(seq[-maxlen:])
        else:
            if padding == 'post':
                res[i, :len(seq)] = np.array(seq)
            else:
                res[i, -len(seq):] = np.array(seq)
    return res

In [13]:
vocab_size = len(id2word)
print(f"Размер словаря: {vocab_size}")

Размер словаря: 12619


## 2. Модели

In [14]:
# Параметры моделей
WINDOW_SIZE = 8
EMB_DIM = 300
NUM_EPOCHS = 20

print(f"Размер окна: {WINDOW_SIZE}")
print(f"Размерность эмбеддингов: {EMB_DIM}")
print(f"Количество эпох: {NUM_EPOCHS}")

Размер окна: 8
Размерность эмбеддингов: 300
Количество эпох: 20


In [15]:
# Генерация датасета для skipgram

def gen_batches_skipgram(sentences, window=5, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)):
                word = sent[i]
                context = sent[max(0, i - window):i] + \
                    sent[i + 1:min(len(sent), i + window + 1)]

                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target_arr = np.array(X_target, dtype='int64')
                        X_context_arr = np.array(X_context, dtype='int64')
                        y_arr = np.array(y, dtype='float32')
                        yield (X_target_arr, X_context_arr), y_arr
                        X_target, X_context, y = [], [], []

In [16]:
# Для CBOW

def gen_batches_cbow(sentences, window=5, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)):
                word = sent[i]
                context = sent[max(0, i - window):i] + \
                    sent[i + 1:min(len(sent), i + window + 1)]

                if context:
                    X_target.append(word)
                    X_context.append(context)
                    y.append(1)

                    X_target.append(np.random.randint(vocab_size))
                    X_context.append(context)
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target_arr = np.array(X_target, dtype='int64')
                        X_context_arr = pad_sequences(
                            X_context, maxlen=window * 2, padding='post', value=0)
                        y_arr = np.array(y, dtype='float32')
                        yield (X_target_arr, X_context_arr), y_arr
                        X_target, X_context, y = [], [], []

### SkipGram с negative sampling

In [18]:
class SkipGramNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)

    def forward(self, target_ids, context_ids):
        t = self.target_emb(target_ids)
        c = self.context_emb(context_ids)
        dot = (t * c).sum(dim=1)
        return dot

### CBOW с negative sampling

In [19]:
class CBOWNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim, window_size):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)
        self.window_size = window_size

    def forward(self, target_ids, context_ids):
        t = self.target_emb(target_ids)
        c = self.context_emb(context_ids)
        c_sum = c.sum(dim=1)
        dot = (t * c_sum).sum(dim=1)
        return dot

### Нахождение похожих слов

In [20]:
from sklearn.metrics.pairwise import cosine_distances


def most_similar(word, embeddings, top_n=10):
    if word not in word2id:
        return f"Слово '{word}' не найдено в словаре"

    similar = [id2word[i] for i in cosine_distances(
        embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:top_n]]
    return similar

## 3. Обучаем SkipGram

In [21]:
model_sg = SkipGramNegSampling(vocab_size, EMB_DIM).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_sg.parameters(), lr=1e-3)

train_gen = gen_batches_skipgram(
    sentences[:int(0.95 * len(sentences))], window=WINDOW_SIZE, batch_size=1000)
valid_gen = gen_batches_skipgram(
    sentences[int(0.95 * len(sentences)):], window=WINDOW_SIZE, batch_size=1000)

steps_per_epoch = 5000
validation_steps = 30

for epoch in tqdm(range(NUM_EPOCHS), desc="Эпохи"):
    model_sg.train()
    epoch_loss = 0.0

    for step in tqdm(range(steps_per_epoch), desc="Шаги", leave=False):
        (X_t, X_c), y = next(train_gen)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer.zero_grad()
        logits = model_sg(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= steps_per_epoch

    model_sg.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_gen)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = model_sg(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()

    val_loss /= validation_steps

    print(
        f"Epoch {
            epoch + 1}/{NUM_EPOCHS} - train loss: {
            epoch_loss:.4f}, val loss: {
                val_loss:.4f}")

Эпохи:   0%|          | 0/20 [00:00<?, ?it/s]

Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/20 - train loss: 5.4505, val loss: 4.9698


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 2/20 - train loss: 3.9419, val loss: 4.1126


...

Epoch 18/20 - train loss: 0.8577, val loss: 0.9616


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 19/20 - train loss: 0.7897, val loss: 0.9749


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 20/20 - train loss: 0.7762, val loss: 0.6533


In [31]:
embeddings_sg = model_sg.context_emb.weight.detach().cpu().numpy()

## 4. Обучаем CBOW

In [23]:
model_cbow = CBOWNegSampling(vocab_size, EMB_DIM, WINDOW_SIZE * 2).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_cbow.parameters(), lr=1e-3)

train_gen = gen_batches_cbow(
    sentences[:int(0.95 * len(sentences))], window=WINDOW_SIZE, batch_size=1000)
valid_gen = gen_batches_cbow(
    sentences[int(0.95 * len(sentences)):], window=WINDOW_SIZE, batch_size=1000)

for epoch in tqdm(range(NUM_EPOCHS), desc="Эпохи"):
    model_cbow.train()
    epoch_loss = 0.0

    for step in tqdm(range(steps_per_epoch), desc="Шаги", leave=False):
        (X_t, X_c), y = next(train_gen)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer.zero_grad()
        logits = model_cbow(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= steps_per_epoch

    model_cbow.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_gen)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = model_cbow(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()

    val_loss /= validation_steps

    print(
        f"Epoch {
            epoch + 1}/{NUM_EPOCHS} - train loss: {
            epoch_loss:.4f}, val loss: {
                val_loss:.4f}")

Эпохи:   0%|          | 0/20 [00:00<?, ?it/s]

Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/20 - train loss: 17.7555, val loss: 13.4106


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 2/20 - train loss: 11.6826, val loss: 10.7289


...

Epoch 18/20 - train loss: 0.7449, val loss: 3.6001


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 19/20 - train loss: 0.6890, val loss: 3.4820


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 20/20 - train loss: 0.6160, val loss: 3.6627


In [34]:
embeddings_cbow = model_cbow.context_emb.weight.detach().cpu().numpy()

# 5. Тестим

In [33]:
# SkipGram

test_words = [
    'компьютер',
    'музыка',
    'война',
    'смерть',
    'язык',
    'лингвистика',
    'самолет',
    'торт']

for word in test_words:
    print(f"{word}: {most_similar(word, embeddings_sg)}")

компьютер: ['компьютер', 'ведьма', 'Sun', 'глинистый', 'воронеж', 'воспринимать', 'уния', 'инновационный', 'лесистый', 'джейми']
музыка: ['музыка', 'она', 'для', 'группа', 'это', 'называть', 'через', 'создавать', 'с', 'иметь']
война: ['война', 'во', 'к', 'с', 'из', 'но', 'он', 'время', 'быть', 'и']
смерть: ['смерть', 'но', 'за', 'со', 'он', 'не', 'как', 'а', 'из', 'время']
язык: ['язык', 'с', 'к', 'в', 'который', 'на', 'из', 'не', 'по', 'и']
лингвистика: ['лингвистика', 'райтман', 'шоколад', 'перевес', 'брилев', 'итого', 'And', 'Hurts', 'Fanuc', 'висконта']
самолет: ['самолет', 'вид', 'главный', 'посвящать', 'боевой', 'оставаться', '18', 'белый', 'гора', 'книга']
торт: ['торт', 'радовицкий', 'ретель', 'осборн', 'пири', 'мегатрон', 'алоис', 'Boys', 'Jordan', 'регби']


In [35]:
# CBOW

for word in test_words:
    print(f"{word}: {most_similar(word, embeddings_cbow)}")

компьютер: ['компьютер', 'набор', 'кельтский', 'карате', 'интерфейс', '2000', 'регулировать', 'замена', 'сервер', 'оптический']
музыка: ['музыка', 'утренний', 'певец', 'и', '1882', 'неизменный', 'калибр', 'трезвучие', 'звучание', 'театр']
война: ['война', 'генрих', 'месть', 'And', '1944', 'варзи', 'полк', 'пасть', 'кампания', 'конфликт']
смерть: ['смерть', 'умирать', 'сын', 'житие', 'президент', 'назик', 'сюжетный', 'граф', '1856', 'брак']
язык: ['язык', 'способность', 'ходовой', 'достижение', 'новелла', 'книга', 'государство', 'село', 'поэт', 'печать']
лингвистика: ['лингвистика', 'коннектикут', 'овраг', 'язык', 'какой', 'нация', 'исток', 'кристаллический', 'мур', 'исследование']
самолет: ['самолет', 'катер', '60', 'максимальный', 'автомобиль', 'экипаж', 'отсек', 'ракета', 'австралия', 'снабжение']
торт: ['торт', 'ординарный', 'принципиально', 'служанка', 'струнный', 'качественный', 'переписывать', 'болезнь', 'справедливый', 'интеллект']


Получается так себе. Кажется, надо удалять стоп-слова, по крайней мере для SkipGramm. CBOW с определением похожих слов справляется ощутимо лучше.

А если попробовать взять эмбеддинги из других матриц?

In [36]:
embeddings_sg = model_sg.target_emb.weight.detach().cpu().numpy()

In [37]:
for word in test_words:
    print(f"{word}: {most_similar(word, embeddings_sg)}")

компьютер: ['компьютер', 'загораться', 'краков', 'передача', 'линкор', 'разрабатывать', 'оценивать', 'показ', 'данные', 'кран']
музыка: ['музыка', 'написать', 'известный', 'автор', 'фильм', 'песня', 'джон', 'основывать', 'роль', 'искусство']
война: ['война', 'после', 'во', 'военный', 'быть', 'войско', 'год', 'г', 'армия', 'в']
смерть: ['смерть', 'сын', 'свой', 'отец', 'его', 'он', 'тот', 'она', 'что', 'не']
язык: ['язык', 'являться', 'другой', 'многий', 'как', 'существовать', 'некоторый', 'автор', 'их', 'слово']
лингвистика: ['лингвистика', 'дисплей', 'пожилой', 'эпидермис', 'коста', 'водяной', '157', 'покупка', 'сигнал', 'совать']
самолет: ['самолет', '27', 'брянский', 'пункт', 'дивизия', 'японский', 'финский', 'ильич', 'го', 'щит']
торт: ['торт', 'упрощенный', 'руины', 'вылетать', 'Alaska', 'понижать', 'вот', 'обстреливать', 'завершение', 'зрительный']


In [38]:
embeddings_cbow = model_cbow.target_emb.weight.detach().cpu().numpy()

In [39]:
for word in test_words:
    print(f"{word}: {most_similar(word, embeddings_cbow)}")

компьютер: ['компьютер', 'Microsoft', 'оперетта', 'плазма', 'памир', 'AT', 'дельта', 'лабораторный', 'выстрел', 'ильич']
музыка: ['музыка', 'дядя', 'инструментальный', 'аквариум', 'игрушка', 'драма', 'автор', 'оркестр', 'творчество', 'иудей']
война: ['война', 'время', 'взятие', 'ветеран', 'изменять', 'совместно', 'действие', 'ответ', 'восстановление', 'приготовление']
смерть: ['смерть', '1170', 'канонический', 'эмилий', 'гостиница', 'муж', 'уходить', 'называться', 'убийство', 'убежденный']
язык: ['язык', 'математический', 'традиция', 'прошедший', 'строка', 'термин', 'конкурент', 'инвестор', 'диалект', 'население']
лингвистика: ['лингвистика', 'психология', 'инсбрук', 'стимулировать', 'вепсский', 'малышка', 'факультет', 'монография', 'акмолинский', 'стремительно']
самолет: ['самолет', 'истребитель', 'аэродром', 'лодка', 'лайнер', 'авиабаза', 'General', 'заказ', 'производить', 'киль']
торт: ['торт', 'масляный', 'юнайтед', 'биохимия', 'охлаждать', 'структурный', 'коростенское', 'грамматик

Я подумал, что для SkipGramm (как минимум) логичнее ожидать правильные эмбеддинги в матрице с таргетным словом, а не контекстом — и да, стало получше, хоть и всё равно не очень. Что интересно — изменение матрицы, откуда мы брали эмбеддинги, для CBOW тоже как будто улучшило результат (и он всё ещё лучше SkipGramm'а). Будем считать, что здесь нужна ещё какая-то умная оптимизация. :)

# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [5]:
import gensim

## 1. Готовим тексты

Те же самые тексты, токенизируем и лемматизируем так же, как и для первого задания.

In [6]:
texts = [preprocess(text) for text in tqdm(wiki)]

  0%|          | 0/20003 [00:00<?, ?it/s]

In [7]:
print(f"Количество текстов для обучения: {len(texts)}")
print(f"Пример первого текста: {texts[0][:10]}")

Количество текстов для обучения: 20003
Пример первого текста: ['новостройка', 'нижегородский', 'область', 'новостройка', 'сельский', 'поселок', 'в', 'дивеевский', 'район', 'нижегородский']


## 2. Обучаем обычную W2V-модель (CBOW)

(потому что он как будто получше похожие слова ищет)

In [8]:
w2v = gensim.models.Word2Vec(
    texts,
    vector_size=500,
    window=8,
    min_count=20,
    max_vocab_size=20000,
    sg=0,
    negative=20,
    epochs=20,
    workers=4
)

In [9]:
print(f"Размер словаря Word2Vec: {len(w2v.wv)}")

Размер словаря Word2Vec: 7321


## 3. Обучаем Fasttext

In [10]:
ft = gensim.models.FastText(
    texts,
    vector_size=500,
    window=8,
    min_count=20,
    # мне лень гуглить, включены ли сюда символьные N-граммы, поэтому пусть
    # будет побольше
    max_vocab_size=50000,
    sg=0,
    negative=20,
    min_n=2,
    max_n=5,
    epochs=20,
    workers=4
)

In [12]:
print(f"Размер словаря FastText: {len(ft.wv)}")

Размер словаря FastText: 16404


## 4. Тестим

In [13]:
# W2V

test_words = [
    'компьютер',
    'музыка',
    'война',
    'смерть',
    'язык',
    'лингвистика',
    'самолет',
    'торт']

for word in test_words:
    try:
        similar = w2v.wv.most_similar(word, topn=10)
        print(f"{word}:")
        for w, score in similar:
            print(f"  {w}: {score:.4f}")
        print()
    except KeyError:
        print(f"{word}: слово не найдено в словаре")

компьютер:
  USB: 0.6055
  процессор: 0.6025
  совместимый: 0.5925
  клавиатура: 0.5782
  PC: 0.5723
  интерфейс: 0.5701
  разъем: 0.5579
  встроенный: 0.5558
  битный: 0.5384
  Apple: 0.5334

музыка:
  аранжировка: 0.5526
  пение: 0.5434
  композитор: 0.5218
  музыкальный: 0.5145
  хоровой: 0.5114
  инструментальный: 0.5059
  мелодия: 0.5037
  джазовый: 0.5033
  вокальный: 0.4888
  репертуар: 0.4813

война:
  вторжение: 0.4432
  кампания: 0.4282
  оккупация: 0.4155
  воевать: 0.3907
  поход: 0.3807
  восстание: 0.3660
  сражение: 0.3432
  бомбардировка: 0.3402
  осада: 0.3386
  революция: 0.3338

смерть:
  гибель: 0.5423
  отъезд: 0.4668
  племянник: 0.3974
  умирать: 0.3947
  внук: 0.3912
  сын: 0.3889
  уход: 0.3860
  самоубийство: 0.3846
  графиня: 0.3809
  похороны: 0.3776

язык:
  диалект: 0.5092
  перевод: 0.4156
  словарь: 0.4126
  языковой: 0.4093
  поэзия: 0.4067
  литература: 0.3966
  славянский: 0.3719
  фольклор: 0.3471
  википедия: 0.3434
  текст: 0.3409

лингвистика: сло

In [15]:
# Fasttext

for word in test_words:
    try:
        similar = ft.wv.most_similar(word, topn=10)
        print(f"{word}:")
        for w, score in similar:
            print(f"  {w}: {score:.4f}")
        print()
    except KeyError:
        print(f"{word}: слово не найдено в словаре")

компьютер:
  компьютерный: 0.8005
  компилятор: 0.6718
  лютер: 0.6098
  шутер: 0.5795
  компиляция: 0.5495
  интерфейс: 0.5440
  PC: 0.5410
  Microsoft: 0.5369
  интер: 0.5228
  тернер: 0.5216

музыка:
  музыкально: 0.8438
  музыкант: 0.8277
  музыкальный: 0.7597
  музыковед: 0.7391
  композитор: 0.5693
  муза: 0.5679
  скрипка: 0.5383
  фортепиано: 0.5091
  композиция: 0.4968
  гармоника: 0.4915

война:
  бойня: 0.5069
  тайна: 0.4902
  хвойный: 0.4331
  войсковой: 0.4082
  оккупация: 0.4010
  войско: 0.3995
  вторжение: 0.3950
  разбойник: 0.3890
  восстание: 0.3857
  военнопленный: 0.3844

смерть:
  паперть: 0.6217
  бессмертие: 0.5912
  смертность: 0.5757
  смертный: 0.5728
  смертельно: 0.5553
  посмертно: 0.5513
  гибель: 0.4887
  четверть: 0.4657
  смертельный: 0.4608
  умирать: 0.4388

язык:
  языковый: 0.7215
  языкознание: 0.7193
  язычник: 0.7055
  языковой: 0.6902
  языческий: 0.5003
  бык: 0.4884
  клык: 0.4689
  англоязычный: 0.4677
  язва: 0.4460
  диалект: 0.4403

линг

Получилось намного лучше, чем в наших самописных велосипедах выше. Видно, что FastText чаще выдаёт похожие по написанию (или просто с общими подсловами) слова, хотя и близкие по смыслу тоже частотны — например, для слова *война*.

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор. 
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%. 

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

tqdm.pandas()

## 1. Загружаем и готовим данные

In [41]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/word_embeddings/labeled.csv')

print(f"Размер датасета: {len(data)}")
print(f"Распределение классов:\n{data['toxic'].value_counts()}")
data.head()

Размер датасета: 14412
Распределение классов:
toxic
0.0    9586
1.0    4826
Name: count, dtype: int64


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


Я буду использовать предобученную модель с леммами, поэтому препроцессинг тот же, что и в первых двух заданиях.

In [42]:
# Лемматизируем
data['lemmas'] = data['comment'].progress_apply(preprocess)

  0%|          | 0/14412 [00:00<?, ?it/s]

## 2. Загружаем предобученную модель

Вместо плохеньких моделей, что мы накрафтили в первых двух заданиях, возьмём (надеюсь) хорошую: "geowac_lemmas_none_fasttextskipgram_300_5_2020". Я выбрал эту модель, а не, например, более новую, обученную на НКРЯ, потому что в тех моделях к лемме присобачили части речи. Тегать 14к примеров тегами UD мне сейчас что-то совсем не хочется, поэтому нашёл модель, которая с леммами, но без POS-тегов — и по метрикам она даже хорошая. Кроме того, это FastText-моделька, а значит и для большего количества слов мы сможем получить эмбеддинги.

In [43]:
!wget https://vectors.nlpl.eu/repository/20/213.zip

!mkdir -p model
!unzip -q 213.zip -d model

--2025-12-27 15:54:09--  https://vectors.nlpl.eu/repository/20/213.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.200, 2001:700:112::200
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1485270300 (1.4G) [application/zip]
Saving to: ‘213.zip’

213.zip             100%[===================>]   1.38G  28.2MB/s    in 52s     

2025-12-27 15:55:01 (27.5 MB/s) - ‘213.zip’ saved [1485270300/1485270300]



In [48]:
ft_pretrained = gensim.models.fasttext.FastTextKeyedVectors.load(
    'model/model.model')

In [49]:
print(f"Размер словаря: {len(ft_pretrained)}")
print(f"Размерность векторов: {ft_pretrained.vector_size}")

Размер словаря: 154923
Размерность векторов: 300


## 3. Векторизация текстов

Просто усредняем. Если ни одного слова нет, придётся занулять.

In [50]:
def text_to_vector(tokens, model):
    vectors = []
    for token in tokens:
        if token in model:
            vectors.append(model[token])

    if vectors:
        return np.mean(vectors, axis=0)
    else:
        # Придётся занулить
        return np.zeros(model.vector_size)

In [51]:
X = np.array([text_to_vector(tokens, ft_pretrained)
             for tokens in tqdm(data['lemmas'])])
y = data['toxic'].values

  0%|          | 0/14412 [00:00<?, ?it/s]

In [52]:
print(f"Размерность X: {X.shape}")
print(f"Размерность y: {y.shape}")

Размерность X: (14412, 300)
Размерность y: (14412,)


## 4. Делим на выборки

90-10

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")

Размер обучающей выборки: 12970
Размер тестовой выборки: 1442


## 5. Обучаем классификатор

In [54]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

LogisticRegression(random_state=42)

## 6. Оцениваем

In [55]:
y_pred = clf.predict(X_test)

In [56]:
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.4f}")

F1-score: 0.8342


In [57]:
print("Classification Report:")
print(
    classification_report(
        y_test,
        y_pred,
        target_names=[
            'non-toxic',
            'toxic']))

Classification Report:
              precision    recall  f1-score   support

   non-toxic       0.90      0.94      0.92       959
       toxic       0.88      0.80      0.83       483

    accuracy                           0.89      1442
   macro avg       0.89      0.87      0.88      1442
weighted avg       0.89      0.89      0.89      1442



Получилось вроде достойно, хотя, кажется, обычный TF-IDF с логрегом давал больше.

В целом, усреднять векторы слов для получения вектора предложения — не самая лучшая идея, особенно в случае статических эмбеддингов.

# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

## 1. Создаём N-граммы

In [58]:
# Функция для создания символьных N-грамм

def ngrammer(raw_string, n=2):
    ngrams = []
    raw_string = ''.join(['<', raw_string, '>'])
    for i in range(0, len(raw_string) - n + 1):
        ngram = ''.join(raw_string[i:i + n])
        if ngram == '<' or ngram == '>':
            continue
        ngrams.append(ngram)
    return ngrams


def split_tokens(tokens, min_ngram_size, max_ngram_size):
    tokens_with_subwords = []
    for token in tokens:
        subtokens = []
        for i in range(min_ngram_size, max_ngram_size + 1):
            if len(token) > i:
                subtokens.extend(ngrammer(token, i))
        tokens_with_subwords.append(subtokens)
    return tokens_with_subwords

In [59]:
class SubwordTokenizer:
    def __init__(self, ngram_range=(1, 1), min_count=5):
        self.min_ngram_size, self.max_ngram_size = ngram_range
        self.min_count = min_count
        self.subword_vocab = None
        self.fullword_vocab = None
        self.vocab = None
        self.id2word = None
        self.word2id = None

    def build_vocab(self, texts):
        unfiltered_subword_vocab = Counter()
        unfiltered_fullword_vocab = Counter()
        for text in tqdm(texts):
            # Используем нашу функцию с лемматизацией
            tokens = preprocess(text)
            unfiltered_fullword_vocab.update(tokens)
            subwords_per_token = split_tokens(
                tokens, self.min_ngram_size, self.max_ngram_size)
            for subwords in subwords_per_token:
                unfiltered_subword_vocab.update(set(subwords))

        self.fullword_vocab = set()
        self.subword_vocab = set()

        for word, count in unfiltered_fullword_vocab.items():
            if count >= self.min_count:
                self.fullword_vocab.add(word)

        for word, count in unfiltered_subword_vocab.items():
            if count >= (self.min_count * 100):
                self.subword_vocab.add(word)

        self.vocab = self.fullword_vocab | self.subword_vocab
        self.id2word = {i: word for i, word in enumerate(self.vocab)}
        self.word2id = {word: i for i, word in self.id2word.items()}

    def subword_tokenize(self, text):
        if self.vocab is None:
            raise AttributeError('Vocabulary is not built!')

        tokens = preprocess(text)
        tokens_with_subwords = split_tokens(
            tokens, self.min_ngram_size, self.max_ngram_size)
        only_vocab_tokens_with_subwords = []

        for full_token, sub_tokens in zip(tokens, tokens_with_subwords):
            filtered = []
            if full_token in self.vocab:
                filtered.append(full_token)
            filtered.extend([subtoken for subtoken in set(
                sub_tokens) if subtoken in self.vocab])
            only_vocab_tokens_with_subwords.append(filtered)

        return only_vocab_tokens_with_subwords

    def encode(self, subword_tokenized_text):
        encoded_text = []
        for token in subword_tokenized_text:
            if not token:
                continue
            encoded_text.append([self.word2id[token[0]]] + [self.word2id[t]
                                for t in set(token[1:]) if t in self.word2id and t != token[0]])
        return encoded_text

    def __call__(self, text):
        return self.encode(self.subword_tokenize(text))

In [60]:
# Делаем словарь
tokenizer = SubwordTokenizer(ngram_range=(2, 5), min_count=30)

tokenizer.build_vocab(wiki)

print(f"Размер полного словаря: {len(tokenizer.vocab)}")
print(f"Размер словаря полных слов: {len(tokenizer.fullword_vocab)}")
print(f"Размер словаря N-грамм: {len(tokenizer.subword_vocab)}")

  0%|          | 0/20003 [00:00<?, ?it/s]

Размер полного словаря: 17167
Размер словаря полных слов: 12942
Размер словаря N-грамм: 4620


## 2. Создание датасета для Fasttext с negative sampling

In [61]:
def gen_batches_ft_neg(
        sentences,
        tokenizer,
        window=5,
        batch_size=1000,
        maxlen=20):
    vocab_size = len(tokenizer.vocab)

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            sent_encoded = tokenizer(sent)

            for i in range(len(sent_encoded)):
                word_with_subtokens = sent_encoded[i]
                context = sent_encoded[max(
                    0, i - window):i] + sent_encoded[i + 1:min(len(sent_encoded), i + window + 1)]

                for context_word_with_subtokens in context:
                    # Положительный пример
                    only_full_word_context_token = context_word_with_subtokens[0]
                    X_target.append(word_with_subtokens)
                    X_context.append(only_full_word_context_token)
                    y.append(1)

                    # Негативный пример
                    X_target.append(word_with_subtokens)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target_arr = pad_sequences(
                            X_target, maxlen=maxlen, padding='post', value=0)
                        X_context_arr = np.array(X_context, dtype='int64')
                        y_arr = np.array(y, dtype='float32')
                        yield (X_target_arr, X_context_arr), y_arr
                        X_target, X_context, y = [], [], []

## 3. Модель Fasttext с negative sampling

In [62]:
class FastTextNegSampling(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.target_emb = nn.Embedding(vocab_size, emb_dim)
        self.context_emb = nn.Embedding(vocab_size, emb_dim)

    def forward(self, target_with_subwords, context_ids):
        # target_with_subwords: (batch, maxlen) - слово + его нграммы
        # context_ids: (batch,) - слово из контекста
        t = self.target_emb(target_with_subwords)  # (batch, maxlen, emb_dim)
        # (batch, emb_dim) - усредняем слово и N-граммы
        t_mean = t.mean(dim=1)
        c = self.context_emb(context_ids)          # (batch, emb_dim)
        # (batch,) - скалярное произведение
        dot = (t_mean * c).sum(dim=1)
        return dot

In [63]:
vocab_size_ft = len(tokenizer.vocab)
emb_dim_ft = 300

print(f"Размер словаря: {vocab_size_ft}")
print(f"Размерность эмбеддингов: {emb_dim_ft}")

Размер словаря: 17167
Размерность эмбеддингов: 300


## 4. Обучаем

In [64]:
model_ft = FastTextNegSampling(vocab_size_ft, emb_dim_ft).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=1e-3)

In [65]:
train_gen_ft = gen_batches_ft_neg(
    wiki[:int(0.95 * len(wiki))], tokenizer, window=7, batch_size=1000, maxlen=20)
valid_gen_ft = gen_batches_ft_neg(
    wiki[int(0.95 * len(wiki)):], tokenizer, window=7, batch_size=1000, maxlen=20)

In [66]:
steps_per_epoch = 5000
validation_steps = 30
num_epochs_ft = 10

for epoch in tqdm(range(num_epochs_ft), desc="Эпохи"):
    model_ft.train()
    epoch_loss = 0.0

    for step in tqdm(range(steps_per_epoch), desc="Шаги", leave=False):
        (X_t, X_c), y = next(train_gen_ft)
        X_t = torch.LongTensor(X_t).to(device)
        X_c = torch.LongTensor(X_c).to(device)
        y_t = torch.FloatTensor(y).to(device)

        optimizer.zero_grad()
        logits = model_ft(X_t, X_c)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= steps_per_epoch

    model_ft.eval()
    val_loss = 0.0
    with torch.no_grad():
        for _ in range(validation_steps):
            (X_t, X_c), y = next(valid_gen_ft)
            X_t = torch.LongTensor(X_t).to(device)
            X_c = torch.LongTensor(X_c).to(device)
            y_t = torch.FloatTensor(y).to(device)

            logits = model_ft(X_t, X_c)
            loss = criterion(logits, y_t)
            val_loss += loss.item()

    val_loss /= validation_steps

    print(
        f"Epoch {
            epoch + 1}/{num_epochs_ft} - train loss: {
            epoch_loss:.4f}, val loss: {
                val_loss:.4f}")

Эпохи:   0%|          | 0/10 [00:00<?, ?it/s]

Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/10 - train loss: 1.1494, val loss: 0.8766


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 2/10 - train loss: 0.6388, val loss: 0.7084


...

Epoch 8/10 - train loss: 0.4359, val loss: 0.4500


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 9/10 - train loss: 0.4364, val loss: 0.4371


Шаги:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 10/10 - train loss: 0.4350, val loss: 0.4349


In [67]:
embeddings_ft = model_ft.target_emb.weight.detach().cpu().numpy()

In [68]:
# Создаем эмбеддинги для полных слов (с учетом N-грамм)
full_word_embeddings_ft = np.zeros((len(tokenizer.fullword_vocab), emb_dim_ft))
id2word_ft = list(tokenizer.fullword_vocab)

for i, word in enumerate(tokenizer.fullword_vocab):
    subwords = tokenizer(word)[0]
    if subwords:
        full_word_embeddings_ft[i] = embeddings_ft[subwords].mean(axis=0)

## 5. Тестим

In [69]:
from sklearn.metrics.pairwise import cosine_distances


def most_similar_ft(word, embeddings, tokenizer, top_n=10):
    if word not in tokenizer.fullword_vocab:
        return f"Слово '{word}' не найдено в словаре"

    subwords = tokenizer(word)[0]
    if not subwords:
        return f"Не удалось получить нграммы для слова '{word}'"

    word_embedding = embeddings[subwords].mean(axis=0)
    similar = [id2word_ft[i] for i in cosine_distances(
        word_embedding.reshape(1, -1), full_word_embeddings_ft).argsort()[0][:top_n]]
    return similar

In [70]:
test_words = [
    'компьютер',
    'музыка',
    'война',
    'смерть',
    'язык',
    'лингвистика',
    'самолет',
    'торт']

for word in test_words:
    result = most_similar_ft(word, embeddings_ft, tokenizer)
    print(f"{word}: {result}")

компьютер: ['компьютер', 'компьютерный', 'композиция', 'компиляция', 'суперкомпьютер', 'компилятор', 'комплект', 'комплектация', 'компенсация', 'компонент']
музыка: ['музыка', 'музыкант', 'музыкально', 'музыкальный', 'концерт', 'язык', 'белка', 'родригес', 'марка', 'языковой']
война: ['война', 'армия', 'год', 'есть', 'быть', 'генерал', '1918', '1943', '1919', '1942']
смерть: ['смерть', 'после', 'говорить', 'свой', 'поддерживать', 'четверть', 'содержать', 'жить', 'смертный', 'знать']
язык: ['язык', 'для', 'и', 'как', 'есть', 'быть', 'в', 'книга', 'этот', 'языковой']
лингвистика: ['лингвистика', 'лингвист', 'лингвистический', 'характеристика', 'стилистика', 'гимнастика', 'статистика', 'журналистика', 'диагностика', 'практика']
самолет: ['самолет', 'пролет', 'фиолетовый', 'пистолет', 'такой', 'болеть', 'полет', 'вертолет', 'случай', 'для']
торт: ['торт', 'корт', 'торпедо', 'торпедный', 'сорт', 'норт', 'ортон', 'торпеда', 'тора', 'тор']


Видно, что в основном выбираются близкие по написанию, или производные слова. Но не всё так плохо, и близкие семантически слова тоже есть: *армия* для слова *война*, *концерт* для слова *музыка* и т.д. В целом результат как будто лучше, чем с моделями в 1 задании, но стоп-слова всё равно надо убирать.

P.S. Я решил оставить свой препроцессинг и работать с леммами, а не со словоформами, поэтому "близкие по форме" заменяются "близкими по написанию" и "производными".